In [1]:
import cobrakbase
%run ../../cobrakbase/AbstractHandleClient.py

cobrakbase 0.2.8


In [2]:
import requests

In [3]:
kbase = cobrakbase.KBaseAPI('K6MO26MQ355EUTJCVPFYIXER4DRS7BWO')

In [4]:
!pwd

/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff


In [5]:
class ShockException(Exception):
    pass

def download_file_from_kbase(token, file_id, file_path, is_handle_ref = 1,
    handle_url = "https://kbase.us/services/handle_service",
    shock_url = "https://kbase.us/services/shock-api",
):
    headers = {'Authorization': 'OAuth ' + token}  
    
    if is_handle_ref == 1:
        hs = AbstractHandle(handle_url, token=token)
        handles = hs.hids_to_handles([file_id])
        file_id = handles[0]['id']
    print(file_path)
    if not os.path.exists(file_path):
        try:
            os.makedirs(file_path)
        except OSError as exc:
            print(exc)
            raise
    elif not os.path.isdir(file_path):
        raise ShockException("file_path: %s must be directory", file_path)
    
    node_url = shock_url + '/node/' + file_id
    r = requests.get(node_url, headers=headers, allow_redirects=True)
    print(r)
    #return r
    if not r.ok:
        errtxt = ('Error downloading file from shock ' +
          'node {}: ').format(file_id)
        try:
            err = json.loads(response.content)['error'][0]
        except Exception:
            # this means shock is down or not responding.
            log("Couldn't parse response error content from Shock: " +
                response.content)
            response.raise_for_status()
        raise ShockException(errtxt + str(err))
    
    resp_obj = r.json()
    size = resp_obj['data']['file']['size']
    if not size:
        raise ShockException('Node {} has no file'.format(shock_id))
    
    node_file_name = resp_obj['data']['file']['name']
    attributes = resp_obj['data']['attributes']
    if os.path.isdir(file_path):
        file_path = os.path.join(file_path, node_file_name)
    with open(file_path, 'wb') as fhandle:
        with requests.get(node_url + '?download_raw', stream=True,headers=headers, allow_redirects=True) as r:
            if not r.ok:
                errtxt = ('Error downloading file from shock ' +
                  'node {}: ').format(file_id)
                try:
                    err = json.loads(response.content)['error'][0]
                except Exception:
                    # this means shock is down or not responding.
                    log("Couldn't parse response error content from Shock: " +
                        response.content)
                    response.raise_for_status()
                raise ShockException(errtxt + str(err))
            for chunk in r.iter_content(1024):
                if not chunk:
                    break
                fhandle.write(chunk)
    return file_path

In [9]:
metagenome_assembly = kbase.get_from_ws('ANME_2C_COP2.fa_assembly.meta.RAST', 92287)

In [12]:
metagenome_assembly.features_handle_ref

'KBH_5593329'

In [32]:
#print(download_file_from_kbase("<TOKEN>",file_id = "KBH_5023369",file_path = "/Users/chenry/"))
response = download_file_from_kbase("K6MO26MQ355EUTJCVPFYIXER4DRS7BWO", file_id = "KBH_5593329", file_path = "/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff")

/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff
<Response [200]>


In [33]:
response.ok

True

In [35]:
?response.raise_for_status

Signature: response.raise_for_status()
Docstring: Raises :class:`HTTPError`, if one occurred.
File:      /usr/local/lib/python3.8/site-packages/requests/models.py
Type:      method


In [30]:
print()

/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff
[Errno 17] File exists: '/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff'
<Response [200]>
/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff/ANME_2C_COP2.fa_assembly.meta.RAST_features.json.gz


In [6]:
import os

In [36]:
import gzip
f = gzip.open('ANME_2C_COP2.fa_assembly.meta.RAST_features.json.gz','rb')
file_content = f.read()

In [41]:
import json
data = json.loads(file_content.decode("utf-8"))

In [44]:
len(data)

4790

In [46]:
{o['type'] for o in data}

{'CDS', 'gene'}

In [34]:
classif = kbase.get_from_ws('ArcheaCLF_decision_tree_classifier_entropy', 77223)

In [35]:
classif.data.keys()

dict_keys(['attribute_data', 'attribute_type', 'class_list_mapping', 'classifier_data', 'classifier_description', 'classifier_handle_ref', 'classifier_id', 'classifier_name', 'classifier_type', 'lib_name', 'number_of_attributes', 'number_of_genomes', 'training_set_ref', 'exclude_dict', 'data_keys', 'info', 'data', 'provenance', 'path', 'creator', 'orig_wsid', 'created', 'epoch', 'refs', 'copied', 'copy_source_inaccessible'])

In [52]:
classif.data['classifier_handle_ref']

'KBH_4997360'

In [54]:
response = download_file_from_kbase("K6MO26MQ355EUTJCVPFYIXER4DRS7BWO", file_id = "KBH_4997360", file_path = "/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff")

/home/fliu/workspace/jupyter/python3/cobrakbase/examples/random_stuff
<Response [200]>


In [14]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import sklearn
print(sklearn.__version__)

0.23.2


In [15]:
with open('ArcheaCLF_decision_tree_classifier_entropy.pickle', 'rb') as fh:
    current_categorizer = pickle.load(fh)

In [27]:
import numpy as np
import pandas as pd

In [50]:
genome_object_data = kbase.ws_client.get_objects2({'objects': [{'ref': '78503/3731/1'}]})['data'][0]['data']
genome1 = kbase.get_from_ws('78503/3731/1')

In [39]:
res = createIndicatorMatrixX(['78503/3731/1'], kbase.ws_client, master_role_list)

In [102]:
def get_list_functional_roles(genome_object_data):
    list_functional_roles = {}

    # figure out where functional roles are kept
    keys_location = genome_object_data.keys()
    if "features" in keys_location:
        location_of_functional_roles = genome_object_data["features"]
        print('features')
    elif "non_coding_features" in keys_location:
        location_of_functional_roles = genome_object_data["non_coding_features"]
        print('non_coding_features')
    elif "cdss" in keys_location:
        location_of_functional_roles = genome_object_data["cdss"]
        print('cdss')
    else:
        raise ValueError("The functional roles are not under features, non_coding_features, or cdss...")

    # either the functional roles are under function or functions (really stupid...)
    keys_function = location_of_functional_roles[0].keys()
    
    function_str = "function" if "function" in keys_function else "functions"
    for functional_role in location_of_functional_roles:
        try:
            for role_to_insert in functional_role[function_str]:
                if " @ " in role_to_insert:
                    list_functional_roles[functional_role['id']] = role_to_insert.split(" @ ")
                elif " / " in role_to_insert:
                    list_functional_roles[functional_role['id']] = role_to_insert.split(" / ")
                elif "; " in role_to_insert:
                    list_functional_roles[functional_role['id']] = role_to_insert.split("; ")
                elif 'hypothetical protein' in role_to_insert:
                    pass
                else:
                    list_functional_roles[functional_role['id']] = {role_to_insert}
        except KeyError as e:
            # print("this is funcitonal role")
            # print(functional_role)
            # print("this is list_functional_roles")
            # print(list_functional_roles)

            # print("apparently some function list just don't have functions...")
            # ^^ this makes no sense...
            pass

    return list_functional_roles
list_functional_roles = get_list_functional_roles(genome_object_data)
print(len(list_functional_roles), len(set(list_functional_roles)))

features
495 495


In [97]:
#list(filter(lambda x: x['id'] == 'K645_RS02840', genome_object_data['features']))[0]

In [98]:
#genome1.features.K645_RS02840.data

In [101]:
for k in list_functional_roles:
    if k in list_functional_roles2:
        f1 = set(list_functional_roles[k])
        f2 = set(list_functional_roles2[k])
        if f1 != f2:
            print(k, f1, '########', f2)

K645_RS00510 {'Bacterial checkpoint controller DisA with nucleotide-binding domain'} ######## {'Diadenylate cyclase spyDAC', 'Bacterial checkpoint controller DisA with nucleotide-binding domain'}
K645_RS00585 {'Succinate-semialdehyde dehydrogenase [NADP+] (EC 1.2.1.79)'} ######## {'Succinate-semialdehyde dehydrogenase [NAD] (EC 1.2.1.24)', 'Succinate-semialdehyde dehydrogenase [NADP+] (EC 1.2.1.79)'}
K645_RS00610 {'NADP-specific glutamate dehydrogenase (EC 1.4.1.4)'} ######## {'NAD-specific glutamate dehydrogenase (EC 1.4.1.2)', 'NADP-specific glutamate dehydrogenase (EC 1.4.1.4)'}
K645_RS02840 {'Molybdenum transport protein, putative'} ######## {'ABC-type Fe3+ transport system protein', 'Molybdenum transport protein, putative'}
K645_RS02945 {'PhnP protein'} ######## {'PhnP protein', 'Metal-dependent hydrolases of the beta-lactamase superfamily I'}


In [111]:
genome1.features.K645_RS02840.add_ontology_term('RAST', 'banada')

In [112]:
genome1.features.K645_RS02840.ontology_terms

{'RAST': ['lol', 'banada']}

In [83]:
#rast.f1('K645_RS02840', genome1.features.K645_RS02840.seq)

In [79]:
import modelseedpy
rast = modelseedpy.RastClient()

In [103]:
list_functional_roles2 = {}
for f in genome1.features:
    if f.functions_unsplit != 'hypothetical protein':
        list_functional_roles2[f.id] = f.functions
print(len(list_functional_roles2), len(set(list_functional_roles2)))

538 538


In [104]:
i1 = len(list_functional_roles)
i2 = len(list_functional_roles2)
for i in range(i1):
    f1 = list_functional_roles[i]
    f2 = list_functional_roles2[i]
    if f1 != f2:
        print(i, f1, '########', f2)

KeyError: 0

In [114]:
def get_list_functional_roles(genome, ontology_term):
    list_functional_roles = []
    for feature in genome.features:
        if ontology_term in feature.ontology_terms:
            list_functional_roles.extend(feature.ontology_terms[ontology_term])
    return list_functional_roles
get_list_functional_roles(genome1, 'RAST')

['lol', 'banada']

In [116]:
def createIndicatorMatrixX(genome_references, ws_client, master_role_list = None, ontology_term='RAST'):
    ref_to_role = {}
    if(master_role_list == None):
        master_role_set = set()
    for genome_ref in genome_references:
        ref_to_role[genome_ref] = get_list_functional_roles(genome, ontology_term)

        if master_role_list is None:
            # keep updateing a set of all functional roles seen so far
            master_role_set |= set(ref_to_role[genome_ref])

    if(master_role_list == None):
        #we are done looping over all genomes
        master_role_list = sorted(list(master_role_set))

        try:
            master_role_list.remove('')
        except:    
            pass
    ref_to_indication = {}

    #make indicator rows for each 
    for genome_ref in genome_references:
        set_functional_roles = set(ref_to_role[genome_ref])
        matching_index = [i for i, role in enumerate(master_role_list) if role in set_functional_roles] 

        indicators = np.zeros(len(master_role_list))
        try:
            indicators[np.array(matching_index)] = 1
        except (IndexError):
            raise IndexError('The genomes or genomeSet that you have submitted wasn’t annotated using the \
                RAST annotation pipeline. Please annotate the genomes via ‘Annotate Microbial Genome’ app \
                (https://narrative.kbase.us/#appcatalog/app/RAST_SDK/reannotate_microbial_genome/release)or \
                genomeSets via Annotate Multiple Microbial Genomes’ app \
                (https://narrative.kbase.us/#appcatalog/app/RAST_SDK/reannotate_microbial_genomes/release) and \
                resubmit the RAST annotated genome/genomeSets into the Predict Phenotype app. (')
        ref_to_indication[genome_ref] = indicators.astype(int)


    indicator_matrix = pd.DataFrame.from_dict(data = ref_to_indication, orient='index', columns = master_role_list).reset_index().rename(columns={"index":"Genome Reference"})

    return indicator_matrix, master_role_list

In [16]:
def createIndicatorMatrix(self, uploaded_df, genome_attribute, master_role_list = None):
    genome_references = uploaded_df["Genome Reference"].to_list()
    ref_to_role = {}
    if(master_role_list == None):
        master_role_set = set()

    for genome_ref in genome_references:
        genome_object_data = self.ws_client.get_objects2({'objects':[{'ref': genome_ref}]})['data'][0]['data']

        #figure out where functional roles are kept
        keys_location = genome_object_data.keys()
        if ("features" in keys_location):
            location_of_functional_roles = genome_object_data["features"]

        elif ("non_coding_features" in keys_location):
            location_of_functional_roles = genome_object_data["non_coding_features"]

        elif ("cdss" in keys_location):
            location_of_functional_roles = genome_object_data["cdss"]

        else:
            raise ValueError("The functional roles are not under features, non_coding_features, or cdss...")

        #either the functional roles are under function or functions (really stupid...)
        keys_function = location_of_functional_roles[0].keys()
        function_str = "function" if "function" in keys_function else "functions"

        list_functional_roles = []
        for functional_role in location_of_functional_roles:
            try:
                role_to_insert = functional_role[function_str][0]
                if " @ " in  role_to_insert:
                    list_functional_roles.extend(role_to_insert.split(" @ "))
                elif " / " in role_to_insert:
                    list_functional_roles.extend(role_to_insert.split(" / "))
                elif "; " in role_to_insert:
                    list_functional_roles.extend(role_to_insert.split("; "))
                elif 'hypothetical protein' in role_to_insert:
                    pass
                else:
                    list_functional_roles.append(role_to_insert)
            except (KeyError):
                # print("this is funcitonal role")
                # print(functional_role)
                # print("this is list_functional_roles")
                # print(list_functional_roles)

                #print("apparently some function list just don't have functions...")
                #^^ this makes no sense...
                pass

        #create a mapping from genome_ref to all of its functional roles
        ref_to_role[genome_ref] = list_functional_roles

        if(master_role_list == None):
            #keep updateing a set of all functional roles seen so far
            master_role_set = master_role_set.union(set(list_functional_roles))

    if(master_role_list == None):
        #we are done looping over all genomes
        master_role_list = sorted(list(master_role_set))

        try:
            master_role_list.remove('')
        except:    
            pass
    ref_to_indication = {}

    #make indicator rows for each 
    for genome_ref in genome_references:
        set_functional_roles = set(ref_to_role[genome_ref])
        matching_index = [i for i, role in enumerate(master_role_list) if role in set_functional_roles] 

        indicators = np.zeros(len(master_role_list))
        try:
            indicators[np.array(matching_index)] = 1
        except (IndexError):
            raise IndexError('The genomes or genomeSet that you have submitted wasn’t annotated using the \
                RAST annotation pipeline. Please annotate the genomes via ‘Annotate Microbial Genome’ app \
                (https://narrative.kbase.us/#appcatalog/app/RAST_SDK/reannotate_microbial_genome/release)or \
                genomeSets via Annotate Multiple Microbial Genomes’ app \
                (https://narrative.kbase.us/#appcatalog/app/RAST_SDK/reannotate_microbial_genomes/release) and \
                resubmit the RAST annotated genome/genomeSets into the Predict Phenotype app. (')
        ref_to_indication[genome_ref] = indicators.astype(int)


    indicator_matrix = pd.DataFrame.from_dict(data = ref_to_indication, orient='index', columns = master_role_list).reset_index().rename(columns={"index":"Genome Reference"})

    return (indicator_matrix, master_role_list)

In [17]:
genome1 = kbase.get_from_ws('GCF_001051995.2.RAST', 'filipeliu:narrative_1606804059500')

In [37]:
master_role_list = classif.attribute_data

In [13]:
current_categorizer

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=1, random_state=0)

In [40]:
#get functional_roles and make indicator matrix
# = self.createIndicatorMatrix(subset_uploaded_df, genome_attribute, master_role_list = master_role_list)
(indicator_matrix, master_role_list) = res
whole_X = indicator_matrix[master_role_list].values

#Make Predictions on uploaded file
predictions_numerical = current_categorizer.predict(whole_X)

In [42]:
class_list_mapping = classif.class_list_mapping
class_list_mapping

{'N': 0, 'P': 1}

In [45]:
inv_map_class_list_mapping = {v: k for k, v in class_list_mapping.items()}
predictions_phenotype = [] #map numerical to phenotype
for numerical in predictions_numerical:
    predictions_phenotype.append(inv_map_class_list_mapping[numerical])
prediction_probabilities_ = current_categorizer.predict_proba(whole_X)
prediction_probabilities = np.max(prediction_probabilities_, axis = 1)

In [ ]:
# 12998/11/2 GramStainKNN